In [1]:
!iperf3 --version

iperf 3.9 (cJSON 1.7.13)
Linux k4tel 5.19.0-38-generic #39~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Fri Mar 17 21:16:15 UTC 2 x86_64
Optional features available: CPU affinity setting, IPv6 flow label, SCTP, TCP congestion algorithm setting, sendfile / zerocopy, socket pacing, authentication


In [2]:
# start the deamon server
!iperf3 -s -B 192.168.0.102 -p 5201 -D

In [3]:
# testing client-server connection
!iperf3 -c 192.168.0.102 -p 5201 -V -t 1

iperf 3.9
Linux k4tel 5.19.0-38-generic #39~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Fri Mar 17 21:16:15 UTC 2 x86_64
Control connection MSS 32768
Time: Fri, 31 Mar 2023 01:23:10 GMT
Connecting to host 192.168.0.102, port 5201
      Cookie: ccym2qehke6425xirs24qky7qo3ttheehcrv
      TCP MSS: 32768 (default)
[  5] local 192.168.0.102 port 60394 connected to 192.168.0.102 port 5201
Starting Test: protocol: TCP, 1 streams, 131072 byte blocks, omitting 0 seconds, 1 second test, tos 0
[ ID] Interval           Transfer     Bitrate         Retr  Cwnd
[  5]   0.00-1.00   sec  4.41 GBytes  37.9 Gbits/sec    0   2.37 MBytes       
- - - - - - - - - - - - - - - - - - - - - - - - -
Test Complete. Summary Results:
[ ID] Interval           Transfer     Bitrate         Retr
[  5]   0.00-1.00   sec  4.41 GBytes  37.9 Gbits/sec    0             sender
[  5]   0.00-1.04   sec  4.41 GBytes  36.5 Gbits/sec                  receiver
CPU Utilization: local/sender 94.0% (3.7%u/90.3%s), remote/receiver 69.4% (11.0%

In [4]:
import subprocess
from parser import *

In [5]:
def client(server_ip, server_port, params=[]):
    prompt = ['iperf3', '-c', server_ip,'-p', server_port] + params
    process = subprocess.Popen(prompt, stdout=subprocess.PIPE, stderr=subprocess.PIPE, encoding='utf-8')
    stdout, stderr = process.communicate()
    return stdout, stderr

In [6]:
# get local machine hostname 
server_ip = subprocess.getoutput('hostname -I').strip()
server_port = "5201"

# client test duration
client_time = 60
# possible client test flags
client_flags = ["--udp", "-V", "-t", str(client_time)]

parser_headers = ['Interval', 'Transfer', 'Bitrate', 'Retr', 'Cwnd']

In [7]:
# test using only time setting in flags
!iperf3 -c 192.168.0.102 -p 5201 -t 1

Connecting to host 192.168.0.102, port 5201
[  5] local 192.168.0.102 port 60408 connected to 192.168.0.102 port 5201
[ ID] Interval           Transfer     Bitrate         Retr  Cwnd
[  5]   0.00-1.00   sec  4.58 GBytes  39.4 Gbits/sec    0   2.12 MBytes       
- - - - - - - - - - - - - - - - - - - - - - - - -
[ ID] Interval           Transfer     Bitrate         Retr
[  5]   0.00-1.00   sec  4.58 GBytes  39.4 Gbits/sec    0             sender
[  5]   0.00-1.04   sec  4.58 GBytes  37.8 Gbits/sec                  receiver

iperf Done.


In [8]:
# using only time setting in flags
result, error = client(server_ip, server_port, client_flags[2:])

if error:
    print(error)
else:
    # standard parser headers in the output
    result_table = parser(result, client_time, parser_headers)
    condition_matches = result_table.loc[(result_table.Transfer > 2) & (result_table.Bitrate > 20), :]
    print(f"Percentage of matching conditions Transfer > 2 & Bitrate > 20:\t" 
        f"{100 * len(condition_matches.index)/len(result_table.index)}%")
    # analysis of the parsed numerical data
    print(result_table.describe())

Headers of the output data: Interval, Transfer, Bitrate, Retr, Cwnd
Percentage of matching conditions Transfer > 2 & Bitrate > 20:	100.0%
       Interval   Transfer    Bitrate  Retr       Cwnd
count      60.0  60.000000  60.000000  60.0  60.000000
mean        1.0   4.568000  39.226667   0.0   4.380833
std         0.0   0.323774   2.781468   0.0   0.608845
min         1.0   3.250000  27.900000   0.0   2.000000
25%         1.0   4.390000  37.700000   0.0   4.560000
50%         1.0   4.625000  39.700000   0.0   4.560000
75%         1.0   4.792500  41.200000   0.0   4.560000
max         1.0   5.090000  43.700000   0.0   4.560000


In [9]:
# test using time setting, extra verbose, and bidirectional testing in flags
!iperf3 -c 192.168.0.102 -p 5201 -V --udp -t 1

iperf 3.9
Linux k4tel 5.19.0-38-generic #39~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Fri Mar 17 21:16:15 UTC 2 x86_64
Control connection MSS 32768
Setting UDP block size to 32768
Time: Fri, 31 Mar 2023 01:24:13 GMT
Connecting to host 192.168.0.102, port 5201
      Cookie: vbhx5wjwfvgwhkghysozlu4dsjq35ozbhsgu
      Target Bitrate: 1048576
[  5] local 192.168.0.102 port 45271 connected to 192.168.0.102 port 5201
Starting Test: protocol: UDP, 1 streams, 32768 byte blocks, omitting 0 seconds, 1 second test, tos 0
[ ID] Interval           Transfer     Bitrate         Total Datagrams
[  5]   0.00-1.00   sec   128 KBytes  1.05 Mbits/sec  4  
- - - - - - - - - - - - - - - - - - - - - - - - -
Test Complete. Summary Results:
[ ID] Interval           Transfer     Bitrate         Jitter    Lost/Total Datagrams
[  5]   0.00-1.00   sec   128 KBytes  1.05 Mbits/sec  0.000 ms  0/4 (0%)  sender
[  5]   0.00-1.04   sec   128 KBytes  1.00 Mbits/sec  0.010 ms  0/4 (0%)  receiver
CPU Utilization: local/sender 7.

In [10]:
# using time setting and bidirectional testing in flags
result, error = client(server_ip, server_port, client_flags)

if error:
    print(error)
else:
    # uncommon parser headers in the output
    result_table = parser(result, client_time)
    condition_matches = result_table.loc[(result_table.Transfer > 2) & (result_table.Bitrate > 20), :]
    print(f"Percentage of matching conditions Transfer > 2 & Bitrate > 20:\t" 
        f"{100 * len(condition_matches.index)/len(result_table.index)}%")
    # analysis of the parsed numerical data
    print(result_table.describe())

Headers of the output data: Interval, Transfer, Bitrate, Total Datagrams
Percentage of matching conditions Transfer > 2 & Bitrate > 20:	0.0%
       Interval  Transfer       Bitrate  Total Datagrams
count      60.0      60.0  6.000000e+01             60.0
mean        1.0     128.0  1.050000e+00              4.0
std         0.0       0.0  2.239184e-16              0.0
min         1.0     128.0  1.050000e+00              4.0
25%         1.0     128.0  1.050000e+00              4.0
50%         1.0     128.0  1.050000e+00              4.0
75%         1.0     128.0  1.050000e+00              4.0
max         1.0     128.0  1.050000e+00              4.0


In [11]:
# shut down the server
!killall iperf3